# Code to generate particle evolution snap-shots

In [ ]:
import numpy as np
import scipy as sp
import scipy.interpolate
import matplotlib.pyplot as plt
import imageio
import cv2

## Load velocity fields

In [ ]:
ForeGroundVx = np.load( 'Vx1.npy' )
ForeGroundVy = np.load( 'Vy1.npy' )

BackGroundVx = np.load( 'Vx2.npy' )
BackGroundVy = np.load( 'Vy2.npy' )

Height, Width = ForeGroundVx.shape

if Height == Width :
    GridSize = Height
else :
    print("ERROR!!")

## Parameters

In [ ]:
NumberForeGroundParticles = 1000
NumberBackGroundParticles = 1000

SpeedRatio = 1/5

BackGroundVx = BackGroundVx * SpeedRatio
BackGroundVy = BackGroundVy * SpeedRatio

In [ ]:
ForeGroundParticleX = (GridSize-1) * np.random.rand( NumberForeGroundParticles )
ForeGroundParticleY = (GridSize-1) * np.random.rand( NumberForeGroundParticles )

BackGroundParticleX = (GridSize-1) * np.random.rand( NumberBackGroundParticles )
BackGroundParticleY = (GridSize-1) * np.random.rand( NumberBackGroundParticles )

plt.plot( ForeGroundParticleX , ForeGroundParticleY , '.r' )
plt.plot( BackGroundParticleX , BackGroundParticleY , '.b' )
plt.axis('equal')
plt.show()

In [ ]:
GridXY = range(GridSize)

ForeGroundVxInterp = sp.interpolate.RegularGridInterpolator( (GridXY , GridXY) , ForeGroundVx )
ForeGroundVyInterp = sp.interpolate.RegularGridInterpolator( (GridXY , GridXY) , ForeGroundVy )

BackGroundVxInterp = sp.interpolate.RegularGridInterpolator( (GridXY , GridXY) , BackGroundVx )
BackGroundVyInterp = sp.interpolate.RegularGridInterpolator( (GridXY , GridXY) , BackGroundVy )

In [ ]:
ForeGroundParticleVx = ForeGroundVxInterp( (ForeGroundParticleY , ForeGroundParticleX) )
ForeGroundParticleVy = ForeGroundVyInterp( (ForeGroundParticleY , ForeGroundParticleX) )

BackGroundParticleVx = BackGroundVxInterp( (BackGroundParticleY , BackGroundParticleX) )
BackGroundParticleVy = BackGroundVyInterp( (BackGroundParticleY , BackGroundParticleX) )

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=300)
fig.add_subplot(2,2,1)
plt.quiver( ForeGroundParticleX , ForeGroundParticleY , ForeGroundParticleVx , ForeGroundParticleVy , units='width' )
fig.add_subplot(2,2,2)
plt.quiver( ForeGroundVx , ForeGroundVy , units='width' )
fig.add_subplot(2,2,3)
plt.quiver( BackGroundParticleX , BackGroundParticleY , BackGroundParticleVx , BackGroundParticleVy , units='width' )
fig.add_subplot(2,2,4)
plt.quiver( BackGroundVx , BackGroundVy , units='width' )
plt.show()

# Particle evolution

In [ ]:
MaxIterations = 100
dt = 10
DisplayIteration = 10

In [ ]:
images = []

for Iteration in range( MaxIterations ):

    # Estimate velocity at the particle position

    ForeGroundParticleVx = ForeGroundVxInterp( (ForeGroundParticleY , ForeGroundParticleX) )
    ForeGroundParticleVy = ForeGroundVyInterp( (ForeGroundParticleY , ForeGroundParticleX) )

    BackGroundParticleVx = BackGroundVxInterp( (BackGroundParticleY , BackGroundParticleX) )
    BackGroundParticleVy = BackGroundVyInterp( (BackGroundParticleY , BackGroundParticleX) )

    # Euler time evolution method 

    ForeGroundParticleX = ForeGroundParticleX + dt * ForeGroundParticleVx
    ForeGroundParticleY = ForeGroundParticleY + dt * ForeGroundParticleVy

    BackGroundParticleX = BackGroundParticleX + dt * BackGroundParticleVx
    BackGroundParticleY = BackGroundParticleY + dt * BackGroundParticleVy

    # Applying periodic boundary condition

    ForeGroundParticleX = np.mod( ForeGroundParticleX , GridSize-1 )
    ForeGroundParticleY = np.mod( ForeGroundParticleY , GridSize-1 )

    BackGroundParticleX = np.mod( BackGroundParticleX , GridSize-1 )
    BackGroundParticleY = np.mod( BackGroundParticleY , GridSize-1 )

    # Writing data

    if np.mod( Iteration , DisplayIteration )==0 :
        fig = plt.figure(figsize=(4,4), dpi=100)
        ax = fig.gca()
        #Image from plot
        ax.axis('off')
        fig.tight_layout(pad=0)

        # To remove the huge white borders
        ax.margins(0)

        plt.plot( ForeGroundParticleX , ForeGroundParticleY , '.b' , markersize=1 )
        plt.plot( BackGroundParticleX , BackGroundParticleY , '.r' , markersize=1 )
        plt.axis('equal')
        plt.axis('off')
        plt.savefig('B.png',bbox_inches='tight',transparent=False,pad_inches = 0)
        # fig.canvas.draw()
        # image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        I = plt.imread( 'B.png' )
        images.append( I )
        plt.show()

imageio.mimsave('movie.gif', images)

In [ ]:
import imageio
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('/path/to/movie.gif', images)